# Machine Learning in Python with Scikit-Learn
<img src="IMG/sk-logo.png">

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

## Scikit-Learn Overview
<img src="IMG/sk-logo.png" width=200>

* dominant Machine Learning Library for Python
* very wide user basis
* very good documentation
* state of the art  implementation 
* unified API 
* full integration in ***NumPy / Pandas*** work flows
* *everything but* **Deep Learning**  


## Scikit-Learn Resources
<img src="IMG/sk-logo.png" width=200>

* Website: https://scikit-learn.org/stable/index.html
* API Reference: https://scikit-learn.org/stable/modules/classes.html
* Tutorial: https://scikit-learn.org/stable/tutorial/index.html

## Scikit-Learn Structure
<img src="IMG/sk-overview.png" width=800>

## Scikit-Learn Structure
<img src="IMG/sk-logo.png" width=200>

***SkLearn*** provides a wide range of ML Algorithms plus methods for:
* loading / accessing data
* data pre-processing
* data selection
* model evaluation 
* model tuning

## Data Access
### Build in Data Sets
***SkLearn*** provides many datasets that are commonly used in Machine Learning teaching and tutorials.
* see full list here: https://scikit-learn.org/stable/datasets/index.html

In [ ]:
from sklearn.datasets import load_iris
X=load_iris()['data'] #vectors of data
Y=load_iris()['target'] #label vector

In [ ]:
type(X)

In [ ]:
X[:20,:]


## Unified API
One key feature of ***SkLearn*** is it's unified API, that allows a very simple exchange ML methods:   

1. create **model instance** for ML Algorithm *A* 
```
model = A( SOME_METHOD_SPECIFIC_PARAMETERS)
```
2. **train** model with data X (and labels Y if we use ***supervised ML***) 
```
model.fit(X) or model.fit(X,Y)
```
3. **inference** of data X_test on our model 
```
pred = model.predict(X_test)
```

### Example: Simple Classification Problem 

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
#generate random date for classification
X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           random_state=1, n_clusters_per_class=1)

In [ ]:
np.shape(X)

In [ ]:
#randomly split into train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
#plot problem
import matplotlib.pyplot as plt
%matplotlib inline
plt.scatter(X[:,0],X[:,1], c=y)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
def plot_surface(clf, X, y, 
                 xlim=(-10, 10), ylim=(-10, 10), n_steps=250, 
                 subplot=None, show=True):
    if subplot is None:
        fig = plt.figure()
    else:
        plt.subplot(*subplot)
        
    xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], n_steps), 
                         np.linspace(ylim[0], ylim[1], n_steps))
    
    if hasattr(clf, "decision_function"):
        z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    else:
        z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
        
    z = z.reshape(xx.shape)
    plt.contourf(xx, yy, z, alpha=0.8, cmap=plt.cm.RdBu_r)
    plt.scatter(X[:, 0], X[:, 1], c=y)
    plt.xlim(*xlim)
    plt.ylim(*ylim)
    
    if show:
        plt.show()

In [ ]:
#train first algorithm: Support Vector Machine
from sklearn.svm import SVC
model = SVC()
model.fit(X_train,y_train)
#plotting model confidence 
plot_surface(model,X_train,y_train, (-2,2), (0,2))

In [ ]:
#now the same problem with a different algorithem: Random Forests
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train,y_train)
#plotting model confidence 
plot_surface(model,X_train,y_train, (-2,2), (0,2))

In [ ]:
#predict
pred = model.predict(X_test)

In [ ]:
#plot inference
plot_surface(model,X_test,pred, (-2,2), (0,2))

In [ ]:
#see if model is correct 
pred==y_test

### Saving and Loading Models
Models are stored via ***pickle***, the ***Python*** serialization library https://docs.python.org/3/library/pickle.html. 


In [ ]:
import pickle 
pickle.dump(model, open( "my_model.p", "wb" ) ) #seave model to fiel
model2 = pickle.load(open( "my_model.p", "rb" ) )#load model from firl
model2.predict(X_test)

## Pre-Processing
***SkLearn*** provides a wide range of pre-processing methods on ***NumPy*** arrays and other input.

In [ ]:
#example scaling data
from sklearn import preprocessing
X_scaled = preprocessing.scale(X_train)

X_scaled[:20,:]                                         

### Scaling
One problem with scaling - as with all other pre-processing methods - is, that we need to find the "right" processing steps based on the **train data** and the also apply it to the **test data**. ***SkLearn*** provides ***Scaler*** models to do this:


In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train)
scaler.mean_ #get model mean                                 


In [ ]:
scaler.scale_ #get scales                                      

In [ ]:
scaler.transform(X_train)                           


### Scaler
There are many different ***Scaler*** available. See [Examples here](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py)

### Normalization
***Normalization*** is the process of scaling **individual samples** to have ***unit norm***. Works just like scaling:

In [ ]:
normalizer = preprocessing.Normalizer(norm='l2').fit(X)

###  Encoding categorical features
Often features are not given as continuous values but categorical. For example a person could have features
```["male", "female"], ["from Europe", "from US", "from Asia"], ["uses Firefox", "uses Chrome", "uses Safari", "uses Internet Explorer"]. 
```
<br><br>
Such features can be efficiently coded as integers, for instance ``["male", "from US", "uses Internet Explorer"]`` could be expressed as [0, 1, 3] while ``["female", "from Asia", "uses Chrome"]`` would be [1, 2, 1].

In [ ]:
#sklearn can do this out-of the box
enc = preprocessing.OrdinalEncoder()
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)  
enc.transform([['female', 'from US', 'uses Safari']])

In [ ]:
enc.transform([['male', 'from Europe', 'uses Firefox']])

#### One-Hot Encoding
Another possibility to convert categorical features to features is to use a ***one-hot*** or dummy encoding. This transforms each categorical feature with **$n$ categories** possible values into **$n$ categories binary features**, with one of them 1, and all others 0.

In [ ]:
enc = preprocessing.OneHotEncoder()
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)  

enc.transform([['female', 'from US', 'uses Safari'],['male', 'from Europe', 'uses Safari']]).toarray()


### Discretization

In [ ]:
#discretize data by dimension
est = preprocessing.KBinsDiscretizer(n_bins=[3, 2], encode='ordinal').fit(X_train)
est.transform(X_test)

### Custom Transformers
***SkLEarn*** also has an easy interface for custom transformation functions

In [ ]:
from sklearn.preprocessing import FunctionTransformer

def myTrans(x):
    return np.log1p(x)

In [ ]:
transformer = FunctionTransformer(myTrans)
transformer.transform(X_train)

## Pipelines
***Pipeline*** can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification. Pipeline serves multiple purposes here:

* Convenience and encapsulation
    
* Joint parameter selection
   
* Safety
   

All estimators in a pipeline, except the last one, must be transformers (i.e. must have a transform method). The last estimator may be any type (transformer, classifier, etc.).

Docs: https://scikit-learn.org/stable/modules/compose.html#pipeline

In [ ]:
from sklearn.pipeline import make_pipeline
normalizer = preprocessing.Normalizer(norm='l2')
model = RandomForestClassifier()
myPipeline = make_pipeline(normalizer,model)

In [ ]:
#now train it
myPipeline.fit(X_train,y_train)

In [ ]:
myPipeline.predict(X_test) 

## Now... hands on!